In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
features=np.load("../input/elmo-embeddings/500-embeddings_elmo.npy")

In [ ]:
features.shape

In [ ]:
features_2=np.load("../input/elmo-embeddings/500-1000-embeddings_elmo.npy")

In [ ]:
features_2.shape

In [ ]:
features_3=np.load("../input/elmo-embeddings/1000-1500-embeddings_elmo.npy")

In [ ]:
features_4=np.load("../input/elmo-embeddings/1500-2000-embeddings_elmo.npy")

In [ ]:
features_5=np.load("../input/elmo-embeddings/2000-2500-embeddings_elmo.npy")

In [ ]:
features_6=np.load("../input/elmo-embeddings/2500-3000-embeddings_elmo.npy")

In [ ]:
features_7=np.load("../input/elmo-embeddings/3000-3500-embeddings_elmo.npy")

In [ ]:
features_8=np.load("../input/elmo-embeddings/3500-4000-embeddings_elmo.npy")

In [ ]:
features_9=np.load("../input/elmo-embeddings/4000-4500-embeddings_elmo.npy")

In [ ]:
features_10=np.load("../input/elmo-embeddings/4500-5000-embeddings_elmo.npy")

In [ ]:
features_11=np.load("../input/elmo-embeddings/5000-5500-embeddings_elmo.npy")

In [ ]:
features_12=np.load("../input/elmo-embeddings/5500-5807-embeddings_elmo.npy")

In [ ]:
features=np.concatenate((features,features_2))

In [ ]:
features.shape

In [ ]:
features=np.concatenate((features,features_3))
features=np.concatenate((features,features_4))
features=np.concatenate((features,features_5))
features=np.concatenate((features,features_6))
features=np.concatenate((features,features_7))
features=np.concatenate((features,features_8))
features=np.concatenate((features,features_9))
features=np.concatenate((features,features_10))
features=np.concatenate((features,features_11))
features=np.concatenate((features,features_12))

In [ ]:
del features_2
del features_3
del features_4
del features_5
del features_6
del features_7
del features_8
del features_9
del features_10
del features_11
del features_12

In [ ]:
import pickle
with open("../input/elmo-embeddings/targets_final.pkl","rb") as f:
    target_list=pickle.load(f)

In [ ]:
target=np.array(target_list)

In [ ]:
target.shape

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=42,test_size=0.3,stratify=Y)

In [ ]:
new_features=features.copy()

In [ ]:
new_features=new_features.sum(axis=1)

In [ ]:
new_features.shape

In [ ]:
X=new_features.reshape(features.shape[0],-1)

In [ ]:
Y=target

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=42,test_size=0.3,stratify=Y)

In [ ]:
import optuna

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import cross_val_score
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 2, 20)
    max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))
    clf = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth)
    return cross_val_score(clf,X_train,Y_train,n_jobs=-1,cv=4).mean()

In [ ]:
study=optuna.create_study(direction='maximize',study_name="random forest study")
study.optimize(objective,n_trials=100)

In [ ]:
clf=RandomForestClassifier(random_state=24,**study.best_trial.params)
clf.fit(X_train,Y_train)

In [ ]:
import joblib
clf=joblib.load("../input/elmo-embeddings/random_forest_with_elmo.pkl")

In [ ]:
clf

In [ ]:
rf_elmo_score=clf.score(X_test,Y_test)

In [ ]:
rf_elmo_score

In [ ]:
class_names=["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"]

In [ ]:
from sklearn.metrics import confusion_matrix
cf=confusion_matrix(Y_test,clf.predict(X_test),normalize="true")

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
fig,ax=plt.subplots(figsize=(10,10))
sns.heatmap(cf, annot=True,cmap="Blues",linewidth=1,linecolor="black",square=True,vmax=1.0)
ax.set_xticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
ax.set_yticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
plt.xticks(rotation=45)

In [ ]:
import joblib

In [ ]:
joblib.dump(clf,"random_forest_with_elmo.pkl")

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 8))
plot_confusion_matrix(clf,X_test,Y_test,normalize="true",display_labels=class_names,ax=ax,cmap="Blues")

In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.model_selection import cross_val_score
def objective(trial):
    svc_c = trial.suggest_loguniform('svc_c', 1, 1e10)
    classifier_obj = SVC(C=svc_c,kernel="rbf",gamma="scale")
    return cross_val_score(classifier_obj,X_train,Y_train,n_jobs=-1,cv=4).mean()

In [ ]:
study=optuna.create_study(direction='maximize',study_name="svm on elmo")
study.optimize(objective,n_trials=100)

In [ ]:
trial=study.best_trial
clf=SVC(C=trial.params["svc_c"],kernel="rbf",gamma="scale")
clf.fit(X_train,Y_train)

In [ ]:
del clf

In [ ]:
clf=joblib.load("../input/elmo-embeddings/svm_with_elmo.pkl")

In [ ]:
clf

In [ ]:
svm_score=clf.score(X_test,Y_test)

In [ ]:
svm_score

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
sns.heatmap(cf, annot=True,cmap="Blues",linewidth=1,linecolor="black",square=True,vmax=1.0)
ax.set_xticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
ax.set_yticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
plt.xticks(rotation=45)

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10, 10))
plot_confusion_matrix(clf,X_test,Y_test,normalize="true",display_labels=class_names,ax=ax)

In [ ]:
joblib.dump(clf,"svm_with_elmo.pkl")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
def objective(trial):
    k=trial.suggest_int('n_neighbors', 30, 70),
    classifier_obj =KNeighborsClassifier(n_neighbors=k)
    return cross_val_score(classifier_obj,X_train,Y_train,n_jobs=-1,cv=4).mean()

In [ ]:
study=optuna.create_study(direction='maximize',study_name="knn_trials")
study.optimize(objective,n_trials=100)

In [ ]:
clf=KNeighborsClassifier(n_neighbors=45)
clf.fit(X_train,Y_train)

In [ ]:
knn_score=clf.score(X_test,Y_test)

In [ ]:
knn_score

In [ ]:
cf=confusion_matrix(Y_test,clf.predict(X_test),normalize="true")
fig,ax=plt.subplots(figsize=(10,10))
sns.heatmap(cf, annot=True,cmap="Blues",linewidth=1,linecolor="black",square=True,vmax=1.0)
ax.set_xticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
ax.set_yticklabels(["BHO","CeeInject","FakeRean","OnLineGames","Renos","Vobfus","Winwebsec"])
plt.xticks(rotation=45)

****Trying to Plot all accuracies together****

In [ ]:
classifiers=["Bert-RF","Bert-SVM","Bert-KNN","Bert-CNN","elmo-RF","elmo-SVM","elmo-KNN","elmo-CNN"]
accuracies=[0.95,0.948,0.89,0.9388,rf_elmo_score,svm_score,knn_score,0.9499]

In [ ]:
accuracy_df=pd.DataFrame({"Classifiers":classifiers,"Accuracy":accuracies})

In [ ]:
accuracy_df

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

In [ ]:
fig,ax=plt.subplots(figsize=(7,10))
ax_=sns.barplot(data=accuracy_df,x="Classifiers",y="Accuracy",color="b",dodge=False,ax=ax)
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', padding=1, rotation=90, size='large')
ax.set(ylim=(0,1))
ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)
plt.xlabel("classifiers")
plt.ylabel("Accuracy")
ax.figure.savefig("all_classifiers.png")